In [1]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O /tmp/cats_and_dogs_filtered.zip

/bin/bash: /home/danieldu/anaconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
wget: /home/danieldu/anaconda3/lib/libuuid.so.1: no version information available (required by wget)
--2023-10-12 10:26:59--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.163.59, 142.251.43.27, 142.251.42.251, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.163.59|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘/tmp/cats_and_dogs_filtered.zip’

/tmp/cats_and_dogs_ 100%[===================>]  65.43M  9.94MB/s    in 11s     

2023-10-12 10:27:09 (6.20 MB/s) - ‘/tmp/cats_and_dogs_filtered.zip’ saved [68606236/68606236]



In [2]:
import os
import zipfile
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow_hub as hub
from tensorflow.keras.layers import InputLayer

local_zip = '/tmp/cats_and_dogs_filtered.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

base_dir = '/tmp/cats_and_dogs_filtered'

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

IMG_SIZE = (224, 224)
INP_SHAPE = (*IMG_SIZE, 3)
BATCH_SIZE = 64

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True)


test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=IMG_SIZE,
        batch_size=20,
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=IMG_SIZE,
        batch_size=20,
        class_mode='binary')

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers.experimental.preprocessing import Resizing


resnet_url = "https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/4"

resnet = Sequential()
resnet.add(Resizing(224, 224, input_shape=INP_SHAPE))
resnet.add(hub.KerasLayer(resnet_url, trainable=False))
resnet.add(Dense(1, activation="sigmoid"))
resnet.summary()

2023-10-12 10:27:11.813049: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-10-12 10:27:11.813254: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-10-12 10:27:11.838002: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-10-12 10:27:11.838196: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-10-12 10:27:11.838347: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from S

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resizing (Resizing)         (None, 224, 224, 3)       0         
                                                                 
 keras_layer (KerasLayer)    (None, 2048)              23564800  
                                                                 
 dense (Dense)               (None, 1)                 2049      
                                                                 
Total params: 23,566,849
Trainable params: 2,049
Non-trainable params: 23,564,800
_________________________________________________________________


In [4]:

resnet.compile(optimizer="rmsprop", loss="binary_crossentropy", metrics=["acc"])

history = resnet.fit(
      train_generator,
      epochs=10,
      validation_data=validation_generator,
      batch_size=32)

Epoch 1/10


2023-10-12 10:27:16.367602: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8500


100/100 [==============================] - 20s 169ms/step - loss: 0.1786 - acc: 0.9305 - val_loss: 0.0438 - val_acc: 0.9850
Epoch 2/10
100/100 [==============================] - 17s 167ms/step - loss: 0.0750 - acc: 0.9725 - val_loss: 0.0530 - val_acc: 0.9810
Epoch 3/10
100/100 [==============================] - 17s 167ms/step - loss: 0.0631 - acc: 0.9705 - val_loss: 0.0288 - val_acc: 0.9920
Epoch 4/10
100/100 [==============================] - 17s 166ms/step - loss: 0.0582 - acc: 0.9770 - val_loss: 0.0263 - val_acc: 0.9910
Epoch 5/10
100/100 [==============================] - 17s 166ms/step - loss: 0.0558 - acc: 0.9790 - val_loss: 0.0307 - val_acc: 0.9920
Epoch 6/10
100/100 [==============================] - 17s 166ms/step - loss: 0.0596 - acc: 0.9800 - val_loss: 0.0321 - val_acc: 0.9910
Epoch 7/10
100/100 [==============================] - 17s 166ms/step - loss: 0.0459 - acc: 0.9840 - val_loss: 0.0260 - val_acc: 0.9920
Epoch 8/10
100/100 [==============================] - 17s 167ms/st

In [5]:
resnet.save("cd.h5")